In [1]:
import numpy as np
import scipy
from scipy.linalg import toeplitz

def postprocess(m, n, b):
    extraction_percent = n / m
    c = np.random.randint(0, 2, m)
    r = np.random.randint(0, 2, n)
    
    T = np.array(toeplitz(c, r))
    out = np.zeros(m)
    running = np.zeros(n)
    
    for i in range(m):
        for j in range(n):
            running[j] = bool(b[j]) & bool(T[i, j])
            if j > 0:
                running[j] = bool(running[j-1]) ^ bool(running[j])
            else:
                continue
        out[i] = int(running[n-1])
    
    return out

In [2]:
from scipy.special import erfc
def monobit_test(b,n):
    b_str = str(b)
    sum = 0
    for x in range(1,len(b_str)):
        if b_str[x] == 1:
            sum += 1
        else:
            sum -= 1
    sobs = abs(sum)/sqrt(n)
    return erfc(sobs/sqrt(2))


In [3]:
from math import floor
from scipy.special import gammainc

def blockfreq_test(b,M,n):
    N = floor(n/M)
    b_str = str(b)
    xobs = 0
    b_str = [b_str[i:i+M] for i in range(0, len(b_str), M)]
    bound = len(b_str)
    if (n%M) != 0:
        bound -= 1
    fracts = np.zeros(bound)
    
    for j in range(0,bound):
        sum = 0
        for k in b_str[j]:
            if k == '1':
                sum+=1
        fracts[j] = sum/M
    sum = 0
    print(sum)
    for i in range(0,N):
        sum += (fracts[i] - 0.5)**2
        print(sum)
    print(sum)
    xobs = 4 * M *sum
    print(xobs)
    return gammainc(N/2, xobs/2)

In [4]:
blockfreq_test('0110011010',3,10)

0
0.027777777777777766
0.05555555555555555
0.08333333333333331
0.08333333333333331
0.9999999999999998


0.19874804309879912

In [2]:
# Importing the required libraries (install in your environment first)
import numpy as np
import pandas as pd
from math import log2, sqrt

# Read in data from your datafile or the provided datafile in this folder to classify QRNG data. 
# Suggested classification strategies include QRNG vs PRNG, QPU vs Simulator, or by individual QPU

#Sample datafile of QRNG (IBM QPUs) vs PRNG data, 12k lines each. Label 1 is QRNG, label 2 is PRNG
data_filePath = 'QRNGvsPRNG_TrainingData.txt'

rng_td = pd.read_table(data_filePath, sep="\s+", header=None)
rng_td.columns =['binary', 'source']
rng_td_nocat = rng_td.copy(deep='true')
rng_features = pd.DataFrame(rng_td['binary'].apply(list).tolist())
rng_td = pd.concat([rng_td.drop(columns='binary'), rng_features], axis=1)


In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

X = rng_td.drop(columns='source').values
y = rng_td['source'].values
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

gb_model = GradientBoostingClassifier(random_state=42)

# Train the model. Define X_train and y_train from your training dataframe using 
# sklearns train_test_split method
gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb_model.predict(X_test)

print(confusion_matrix(y_pred_gb,y_test))
# Calculate the accuracy of the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Gradient Boosting Accuracy: ", accuracy_gb)


[[1205 1111]
 [1186 1298]]
Gradient Boosting Accuracy:  0.5214583333333334


In [7]:
data_filePath2 = '100 qubits QRNG Output_Training Data.txt'
rng2_td = pd.read_table(data_filePath2, sep="\s+", header=None)

rng2_features = pd.DataFrame(rng2_td.loc[0].apply(list).tolist())

y_pred2_gb = gb_model.predict(rng2_features.to_numpy())

# Calculate the accuracy of the Gradient Boosting model
accuracy2_gb = accuracy_score(np.ones(100), y_pred2_gb)
print("Gradient Boosting Accuracy: ", accuracy2_gb)

Gradient Boosting Accuracy:  0.54


In [8]:
from qiskit import QuantumCircuit, transpile
HGateCircuit = QuantumCircuit(8)
def transmit(A,GA):
    for i in range(8):
        if(GA[i] == 0): #B[i] = 0, measure in z, B[i] = 1, measure in x
            if A[i] == 0:
                HGateCircuit.h(i);    
#BB84
#a: (4 + δ)n, b: (4 + δ)n
A = np.random.randint(0, 2, 8) #PRNG
GA = np.random.randint(0, 2, 8) #PRNG
B = np.random.randint(0, 2, 8) #PRNG
GB = np.random.randint(0, 2, 8) #PRNG
transmit(A,GA)
transmit(B,GB)

HGateCircuit.draw()


q_0: ──────────
     ┌───┐     
q_1: ┤ H ├─────
     ├───┤┌───┐
q_2: ┤ H ├┤ H ├
     ├───┤└───┘
q_3: ┤ H ├─────
     └───┘     
q_4: ──────────
               
q_5: ──────────
     ┌───┐     
q_6: ┤ H ├─────
     └───┘     
q_7: ──────────

In [9]:
def calculate_max_entropy(bit_string):
    probabilities = np.bincount([int(bit) for bit in bit_string]) / len(bit_string)
    entropy = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
    return entropy

In [10]:
def calculate_min_entropy(bit_string):
    probabilities = np.bincount([int(bit) for bit in bit_string]) / len(bit_string)
    maxp = np.max(probabilities)
    min_entropy = - np.log2(maxp)
    return min_entropy

In [11]:
summe = 0
summax = 0
for i in range (0,1000):
    summe += calculate_min_entropy(np.random.randint(0, 2, 100))
    summax += calculate_max_entropy(np.random.randint(0, 2, 100))

avgme = summe/1000
avgmax = summax/1000
print(avgme)
print(avgmax)

0.8913225229776296
0.9929628055831883


In [12]:
summe = 0
summax = 0

for i in range (0,100):
    summe += calculate_min_entropy((rng2_td[i][0]))
    summax += calculate_max_entropy((rng2_td[i][0]))
for i in range (0,900):
    summe += calculate_min_entropy((rng_td[0][i]))
    summax += calculate_max_entropy((rng_td[0][i]))

avgme = 10*summe/1000
avgmax = 10*summax/1000

print(avgme)
print(avgmax)

0.8959892302251026
0.9931296313434375


In [6]:
rng3_td = pd.read_table('updated (1).txt', sep="\s+", header=None)
rng3_features = pd.DataFrame(rng3_td.loc[0].apply(list).tolist())
for i in range (0,1000):
    summe += calculate_min_entropy((rng3_td[i][0]))
    summax += calculate_max_entropy((rng3_td[i][0]))

TypeError: 'int' object is not iterable

In [5]:
failures_mono = 0
success_mono = 0
for i in range (0,100):
    tv = monobit_test((rng2_td[i][0]), 100)
    if tv < 0.01:
        failures_mono += 1
    else:
        success_mono += 1
print(failures_mono)
print(success_mono)


NameError: name 'monobit_test' is not defined

In [14]:
failures_block = 0
success_block = 0
for i in range (0,100):
    tv = blockfreq_test((rng2_td[i][0]), 3, 100)
    if tv < 0.01:
        failures_block += 1
    else:
        success_block += 1
print(failures_block)
print(success_block)

0
0.25
0.2777777777777778
0.3055555555555556
0.33333333333333337
0.5833333333333334
0.8333333333333334
0.8611111111111112
0.888888888888889
0.9166666666666667
1.1666666666666667
1.4166666666666667
1.6666666666666667
1.9166666666666667
1.9444444444444444
1.972222222222222
1.9999999999999998
2.0277777777777777
2.0555555555555554
2.083333333333333
2.1111111111111107
2.1388888888888884
2.166666666666666
2.1944444444444438
2.2222222222222214
2.249999999999999
2.499999999999999
2.527777777777777
2.5555555555555545
2.583333333333332
2.833333333333332
2.86111111111111
2.8888888888888875
2.916666666666665
2.916666666666665
34.999999999999986
0
0.25
0.2777777777777778
0.3055555555555556
0.5555555555555556
0.5833333333333334
0.8333333333333334
0.8611111111111112
0.888888888888889
0.9166666666666667
1.1666666666666667
1.1944444444444444
1.222222222222222
1.2499999999999998
1.2777777777777775
1.3055555555555551
1.3333333333333328
1.3611111111111105
1.3888888888888882
1.6388888888888882
1.6666666666